In [21]:
import numpy as np
import pandas as pd
import plotly.express as px
from operator import methodcaller
import os
from sklearn import linear_model
from util import *
import math
import pathlib

In [22]:
subfolders = [ f.name for f in os.scandir(os.getcwd()) if f.is_dir() ]
recallibrated_dir = 'recallibrated_data'
exclusions = ['data_validate', 'recallibrated_data', 'pilot_testing']
error_df = pd.DataFrame(columns=['Name', 'w1', 'w2', 'w3', 's4', 'b5'])
error_df_path = recallibrated_dir + '/' +'error_df.csv'

if os.path.exists(error_df_path):
    error_df = pd.read_csv(error_df_path)

for folder in subfolders:
    desktop = pathlib.Path(folder)
    files = [f.name for f in desktop.iterdir() if f.is_file()]
    data_files = []
    callibration_file = 'not_found'
    
    for f in files:
        if "callibrate" in f:
            callibration_file = folder+"/"+f
        elif "csv" in f:
            data_files.append(folder+"/"+f)

    if callibration_file == 'not_found' or folder in exclusions:
        continue

    if os.path.exists(recallibrated_dir + '/' + files[0][:-4] + '_recal.csv'):        
        if not folder in error_df['Name'].values:
            errors_in_scene = {}
            for f in files:
                if "callibrate" in f:
                    continue
                df = pd.read_csv(recallibrated_dir + '/' + f[:-4] + '_recal.csv')
                if 'w1' in f:
                    errors_in_scene['w1'] = spatial_euc_errors(df)
                elif 'w2' in f:
                    errors_in_scene['w2'] = spatial_euc_errors(df)
                elif 'w3' in f:
                    errors_in_scene['w3'] = spatial_euc_errors(df)
                elif 's4' in f:
                    errors_in_scene['s4'] = spatial_euc_errors(df)
                elif 'b5' in f:
                    errors_in_scene['b5'] = spatial_euc_errors(df)

            error_df.loc[len(error_df.index)] = [folder, errors_in_scene['w1'], errors_in_scene['w2'], errors_in_scene['w3'], errors_in_scene['s4'], errors_in_scene['b5']]
        continue
    
    vis_angles_callib = vis_angles_from_file(callibration_file)
    X1 = vis_angles_callib[['gaze_vis_x','gaze_vis_y']]
    y1 = vis_angles_callib['target_vis_x']

    X2 = vis_angles_callib[['gaze_vis_x','gaze_vis_y']]
    y2 = vis_angles_callib['target_vis_y']
    x_regr = linear_model.LinearRegression()
    x_regr.fit(X1.values, y1.values)
    coeffs1 = x_regr.coef_

    y_regr = linear_model.LinearRegression()
    y_regr.fit(X2.values, y2.values)
    coeffs2 = y_regr.coef_
    if not os.path.exists(recallibrated_dir):
        os.makedirs(recallibrated_dir)

    print([x_regr.coef_, y_regr.coef_])

    errors_in_scene = {}

    for f in data_files:
        df = pd.read_csv(f)
        indexExpStarts = 0
        for index, row in df.iterrows():
            if row['PathIDX'] != 99:
                indexExpStarts = index
                break

        indexExpStarts = max(0, indexExpStarts-500)
        df = df.drop(range(indexExpStarts))
        df = df[df["left_right_eye_is_blinking"].str.contains("True") == False]
        df['seconds'] = df['seconds'].apply(lambda x: x-df['seconds'].iat[0])
        df['frame'] = df['frame'].apply(lambda x: x-df['frame'].iat[0])

        df.index = np.arange(0, len(df))
        df['gaze_x_recal'] = df.apply(lambda row: x_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)
        df['gaze_y_recal'] = df.apply(lambda row: y_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)

        df = df[['frame','PathIDX','seconds','gaze_vis_x','gaze_vis_y','gaze_x_recal','gaze_y_recal', 'target_vis_x', 'target_vis_y']]
        f = f.replace(folder, '')
        df.to_csv(recallibrated_dir+f[:-4]+'_recal.csv')

        if 'w1' in f:
            errors_in_scene['w1'] = spatial_euc_errors(df)
        elif 'w2' in f:
            errors_in_scene['w2'] = spatial_euc_errors(df)
        elif 'w3' in f:
            errors_in_scene['w3'] = spatial_euc_errors(df)
        elif 's4' in f:
            errors_in_scene['s4'] = spatial_euc_errors(df)
        elif 'b5' in f:
            errors_in_scene['b5'] = spatial_euc_errors(df)

        if len(errors_in_scene) == 5:
            error_df.loc[len(error_df.index)] = [folder, errors_in_scene['w1'], errors_in_scene['w2'], errors_in_scene['w3'], errors_in_scene['s4'], errors_in_scene['b5']]

        

In [23]:
error_df.to_csv(error_df_path, index= False)
